# Utils

**Импорты библиотек:**

In [1]:
!pip install tqdm==4.66.5
!pip install torch==2.4.1+cu121
!pip install torchvision==0.19.1+cu121
!pip install transformers==4.44.2
!pip install comet_ml==3.47.6
!pip install triton==3.1.0
!pip install torchattacks==3.5.1
!pip install jax==0.4.38
!pip install jaxlib==0.4.38
!pip install pytorch_optimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.6/710.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.3/980.3 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.2.0
    Uninstalling python-box-7.2.0:
      Successfully uninstalled python-box-7.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [2]:
import random

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import os

import comet_ml

import torch
import torchvision
from torchvision.transforms import v2
import torchattacks
from transformers import BeitConfig, BeitForMaskedImageModeling
import torch.nn as nn

from transformers import AdamW
import torch.optim as optim
from transformers import Dinov2ForImageClassification
from transformers import AutoModel
from copy import deepcopy

**Utils:**

**Функция для фиксирования сида:**

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

**Additional utils:**

In [4]:
def get_root(env):
    # supports env == "colab", env == "kaggle"
    if env == "kaggle":
        return "/kaggle/working/"
    else:
        return "./"

In [5]:
# this augmentation first converts PIL image to Tensor,
# then casts resulting tensor to type=torch.float32,
# finally normalizes the image

default_aug = v2.Compose([v2.PILToTensor(),
                          v2.ToDtype(torch.float32, scale=True),
                          v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

In [6]:
# This function is used later in DataLoaders to fix the seed setting bugs when using several workers

def worker_init_fn(worker_id):
    torch_seed = torch.initial_seed() % (2**32)
    random.seed(torch_seed)
    np.random.seed(torch_seed)

**Датасет + DataLoaders:**

In [7]:
def get_CIFAR10_data(batch_size, train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    print("Prepare Test Set:  ✅")
    return data_train, data_test

In [8]:
def get_CIFAR10_dataloaders(batch_size, train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    dataloader_train = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Test Set:  ✅")
    return dataloader_train, dataloader_test

**Код цикла предобучения/дообучения SSL-метода:**

In [9]:

def pretrain_beit(model, processor, train_loader, epochs, optimizer, scaler, scheduler=None, device="cuda"):
    model.train()
    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}")
        for batch_idx, (images, _) in enumerate(train_loader):
            images = images.to(device, memory_format=torch.channels_last)

            inputs = processor(images, return_tensors="pt", do_rescale=False).to(device)
            pixel_values = inputs["pixel_values"]
            
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(pixel_values=pixel_values, labels=labels)
                loss = outputs.loss
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            if batch_idx % 100 == 0:
                print(f"Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")
        
        if scheduler:
            scheduler.step()
        print(f"Epoch {epoch} completed. Current LR: {scheduler.get_last_lr()[0]}")
    
    print("Pretraining Complete!")
    return model

In [10]:
def pretrain(model, train_loader, optimizer, criterion, epochs=10):
  model.train()
  for epoch in range(epochs):
      running_loss = 0.0
      for images, labels in train_loader:
          images = images.to(device)
          labels = labels.to(device)

          # Zero gradients
          optimizer.zero_grad()

          # Extract features using pre-trained DINOv2 backbone
          with torch.no_grad():
              features = model.dino(images)

          # Forward pass through the custom classifier head
          outputs = model.classifier(features)

          # Calculate the loss
          loss = criterion(outputs, labels)

          # Backpropagate the loss and optimize
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      # Print loss for the epoch
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

In [11]:
comet_ml.login()

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


Comet API key:  ········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
NUM_EPOCHS = 50
BATCH_SIZE = 128
exp_name = "BEiT_Pretrain"

In [14]:
from transformers import BeitForImageClassification, BeitConfig, AutoImageProcessor, BeitForMaskedImageModeling

In [15]:
# # random seed
# seed_everything(0)

# # device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# #model
# beit_config = BeitConfig.from_pretrained("microsoft/beit-base-patch16-224")
# model = BeitForMaskedImageModeling.from_pretrained(
#     "microsoft/beit-base-patch16-224", config=beit_config
# )
# model.to(device)

# optimizer = AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)
# scaler = torch.cuda.amp.GradScaler()
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# processor = AutoImageProcessor.from_pretrained("microsoft/beit-base-patch16-224")

# # pretrained_model = pretrain_beit(model, processor, train_loader, epochs=NUM_EPOCHS, optimizer=optimizer, scaler=scaler, scheduler=scheduler, device=device)

# train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE)

# torch.save(pretrained_model.state_dict(), "beit_cifar10_pretrained.pth")
# print("Pretrained model saved!")

**Downstream-модель:**

In [16]:
class DownstreamModel(torch.nn.Module):
    def __init__(self, encoder, head):
        super(DownstreamModel, self).__init__()
        self.encoder = encoder
        self.head = head

    def forward(self, x):
        enc_features = self.encoder(x)
        logits = self.head(enc_features)
        return logits

**Код для обычного/adversarial Fine-Tuning'а (на одном GPU):**

In [17]:
def eval_epoch(model, loader, attack=None):
    val_loss = 0.
    val_acc = 0.

    model.eval()

    for data, target in tqdm(loader):
        data = data.to(memory_format=torch.channels_last) # this line is for a speedup (used in a combination with amp)
        data = data.to(device)
        target = target.to(device)

        if attack is not None:
            # perform adversarial attack
            with torch.enable_grad():
                data = attack(data, target)
        
        logits = model(data)
        loss = torch.nn.functional.cross_entropy(logits, target)

        val_loss += (loss.item() * data.shape[0])
        val_acc += (torch.argmax(logits, dim=1) == target).sum().item()

    val_loss /= len(loader.dataset)
    val_acc /= len(loader.dataset)
    return val_loss, val_acc


def train_epoch(model, optimizer, loader, scaler, epoch, exp, is_attack_used=False):
    train_loss = 0.
    train_acc = 0.

    step = (epoch - 1) * len(loader)

    model.train()

    for batch_idx, (data, target) in enumerate(tqdm(loader)):
        data = data.to(memory_format=torch.channels_last) # this line is for a speedup (used in a combination with amp)
        data = data.to(device)
        target = target.to(device)

        if is_attack_used != "no":
            if is_attack_used == "BIM":
                attack = torchattacks.BIM(model, eps=(4 / 255), alpha=(2 / 255), steps=5)
            else:
                attack = torchattacks.PGD(model, eps=(8 / 255), alpha=(2 / 255), steps=20)
            # perform adversarial attack
            data = attack(data, target)

        optimizer.zero_grad()

        with torch.autocast("cuda", dtype=torch.float16, enabled=True):
            logits = model(data)
            loss = torch.nn.functional.cross_entropy(logits, target)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        with torch.no_grad():
            train_loss += loss.item() * data.shape[0]
            train_acc += (torch.argmax(logits, dim=1) == target).sum().item()

        if exp is not None:
            exp.log_metrics({"train loss (by steps)": loss.item()}, step=(step + batch_idx))

    train_loss /= len(loader.dataset)
    train_acc /= len(loader.dataset)
    return train_loss, train_acc, scaler


def train(model, exp_name, optimizer, epochs, train_loader, val_loader, is_train_attack_used="no", scheduler=None, exp=None, env="colab"):
    best_val_acc = 0.
    best_val_acc_epoch = None
    best_model_state_filepath = None

    val_attacks_names = ["No Attack"]

    scaler = torch.GradScaler("cuda")

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}:\n")
        metrics = dict()
        
        train_loss, train_acc, scaler = train_epoch(model, optimizer, train_loader, scaler, epoch, exp, is_attack_used=is_train_attack_used)

        val_attacks = [None]
        
        for it in range(len(val_attacks)):
            val_loss, val_acc = eval_epoch(model, val_loader, attack=val_attacks[it])
            metrics[val_attacks_names[it]] = {"Loss": val_loss, "Accuracy": val_acc}

        if scheduler is not None:
            scheduler.step()

        print("\n")
        print(" Train:")
        print(f"  Loss: {train_loss}, Accuracy: {train_acc}")
        print(" Validation:")
        for it in metrics.items():
            print(f"  Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")


        if exp is not None:
            exp.log_metrics(
                   {"train loss": train_loss,
                    "train accuracy": train_acc},
                    step=(epoch * len(train_loader)))
            for it in metrics.items():
                exp.log_metrics(
                    {f"validation loss ({it[0]})": it[1]['Loss'],
                     f"validation accuracy ({it[0]})": it[1]['Accuracy']},
                     step=(epoch * len(train_loader)))

        val_acc = metrics["No Attack"]["Accuracy"]
        
        if val_acc > best_val_acc:
            print("🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉")
            best_val_acc = val_acc
            best_val_acc_epoch = epoch
            filepath = f"{get_root(env)}best_weights_{exp_name}_epoch{best_val_acc_epoch}.pth"
            torch.save(model.state_dict(), filepath)
            print("Save State of the Best Model: ✅")
            if best_model_state_filepath is not None:
                os.remove(best_model_state_filepath)
                print("Delete State File of the Previous Best Model: ✅")
            best_model_state_filepath = filepath

        print('-----------------------------------------------------------------\n')

    if exp is not None:
        exp.log_model(name=best_model_state_filepath[len(get_root(env)):-4],
                      file_or_folder=best_model_state_filepath)


def test(model, loader, env="colab"):
    attacks_names = ["No Attack", "i-FGSM", "PGD", "DI-FGSM"]
    
    metrics = dict()

    attacks = [None, 
               torchattacks.BIM(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.PGD(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.DIFGSM(model, eps=(8 / 255), alpha=(2 / 255), steps=10, resize_rate=0.9, diversity_prob=0.5)
              ]
    
    for it in range(len(attacks)):
        loss, acc = eval_epoch(model, loader, attack=attacks[it])
        metrics[attacks_names[it]] = {"Loss": loss, "Accuracy": acc}

    print("\n")
    print("Final Metrics:")
    for it in metrics.items():
        print(f" {it[0]}:")
        print(f"   Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")

**Примерный код, как с помощью весов получить энкодер модели + создание downstream-модели (нужно для SSL-методов):**

# Эксперименты

In [18]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [19]:
comet_ml.login()

In [20]:
from transformers import BeitConfig, BeitModel, AutoImageProcessor

**BEiT  (Adversarial SFT with i-FGSM, without patch resizing)**

In [27]:
class BEiTClassifier(nn.Module):
    def __init__(self, encoder, num_classes):
        super(BEiTClassifier, self).__init__()
        self.encoder = encoder

        head_hidden_dim = self.encoder.config.hidden_size * 2

        self.head = nn.Sequential(
            nn.Linear(self.encoder.config.hidden_size, head_hidden_dim),
            nn.GELU(),
            nn.Linear(head_hidden_dim, num_classes)
        )

    def forward(self, x):
        enc_features = self.encoder(x).last_hidden_state[:, 0, :]
        logits = self.head(enc_features)
        return logits

In [28]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "BEIT_SFT_IFGSM"

In [29]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/beit-base-patch16-224")

beit_config = BeitConfig.from_pretrained("microsoft/beit-base-patch16-224")
encoder = BeitModel.from_pretrained("microsoft/beit-base-patch16-224", config=beit_config)

model = BEiTClassifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
print()

In [30]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="kaggle")

# choose optimizer and scheduler (if needed)
# optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [31]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="BIM", scheduler=scheduler, exp=exp, env="kaggle")

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/denterry/ssl-adversarial/d7e64e5c4f514252839cdc2136c6ad7f

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:02<00:00, 26.42it/s]




 Train:
  Loss: 2.284415138053894, Accuracy: 0.3067
 Validation:
  Loss: 1.4032121173858643, Accuracy: 0.4862
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:02<00:00, 26.53it/s]


...

100%|██████████| 79/79 [00:02<00:00, 27.37it/s]




 Train:
  Loss: 0.9613922851753235, Accuracy: 0.64832
 Validation:
  Loss: 0.9213652102470398, Accuracy: 0.6702
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------

Epoch 5:



100%|██████████| 79/79 [00:02<00:00, 26.58it/s]




 Train:
  Loss: 0.801125520439148, Accuracy: 0.7096
 Validation:
  Loss: 0.8753922724723816, Accuracy: 0.6907
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : BEIT_SFT_IFGSM
COMET INFO:     url                   : https://www.comet.com/denterry/ssl-adversarial/d7e64e5c4f514252839cdc2136c6ad7f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (628.3218994140625, 1131036.25)
COMET INFO:     train accuracy [5]                  : (0.3067, 0.7096)
COMET INFO:     train loss (by steps) [1955]        : (0.6003427505493164, 19.47885513305664)
COMET INFO:     train loss [5]                      : (0.801125520439148, 2.284415138053894)
COMET INFO:     validation accuracy (No Attack) [5] : (0.4862, 0.6907)
COMET INFO:     validation loss (No Attack) [5]     : (0.875392272

In [32]:
test(model, test_loader, env="kaggle")

100%|██████████| 79/79 [00:45<00:00,  1.75it/s]



Final Metrics:
 No Attack:
   Loss: 0.8753922724723816, Accuracy: 0.6907
 i-FGSM:
   Loss: 1.4572285566329957, Accuracy: 0.49
 PGD:
   Loss: 3.072203533935547, Accuracy: 0.1481
 DI-FGSM:
   Loss: 2.942058819961548, Accuracy: 0.1729


**BEiT  (Adversarial SFT with PGD, without patch resizing)**

In [22]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "BEIT_SFT_PGD"

In [23]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/beit-base-patch16-224")

beit_config = BeitConfig.from_pretrained("microsoft/beit-base-patch16-224")
encoder = BeitModel.from_pretrained("microsoft/beit-base-patch16-224", config=beit_config)

model = BEiTClassifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
print()

preprocessor_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/350M [00:00<?, ?B/s]

In [24]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="kaggle")

# choose optimizer and scheduler (if needed)
# optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

100%|██████████| 170498071/170498071 [00:08<00:00, 20936847.45it/s]


Extracting /kaggle/working/cifar-10-python.tar.gz to /kaggle/working/
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [25]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="PGD", scheduler=scheduler, exp=exp, env="kaggle")

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/denterry/ssl-adversarial/e5ee3fdf600d4b218bafcbae632630cf



Epoch 1:



100%|██████████| 79/79 [00:03<00:00, 25.76it/s]




 Train:
  Loss: 2.8569624488067626, Accuracy: 0.18112
 Validation:
  Loss: 2.025092115402222, Accuracy: 0.2639
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:02<00:00, 26.84it/s]


...

100%|██████████| 79/79 [00:02<00:00, 26.43it/s]




 Train:
  Loss: 1.8416591625595093, Accuracy: 0.31878
 Validation:
  Loss: 2.050991609954834, Accuracy: 0.2799
-----------------------------------------------------------------

Epoch 5:



100%|██████████| 79/79 [00:03<00:00, 26.14it/s]




 Train:
  Loss: 1.79063148563385, Accuracy: 0.33714
 Validation:
  Loss: 1.9871357917785644, Accuracy: 0.295
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : BEIT_SFT_PGD
COMET INFO:     url                   : https://www.comet.com/denterry/ssl-adversarial/e5ee3fdf600d4b218bafcbae632630cf
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (838.3046875, 1473799.75)
COMET INFO:     train accuracy [5]                  : (0.18112, 0.33714)
COMET INFO:     train loss (by steps) [1955]        : (1.5661678314208984, 22.935888290405273)
COMET INFO:     train loss [5]                      : (1.79063148563385, 2.8569624488067626)
COMET INFO:     validation accuracy (No Attack) [5] : (0.2639, 0.295)
COMET INFO:     validation loss (No Attack) [5]     : (1.987135791778564

In [26]:
test(model, test_loader, env="kaggle")

100%|██████████| 79/79 [00:45<00:00,  1.75it/s]



Final Metrics:
 No Attack:
   Loss: 1.9871357917785644, Accuracy: 0.295
 i-FGSM:
   Loss: 2.1059266143798827, Accuracy: 0.2475
 PGD:
   Loss: 1.792205802154541, Accuracy: 0.3322
 DI-FGSM:
   Loss: 1.7752964181900024, Accuracy: 0.3409


**BEiT  (Adversarial SFT without attacks)**

In [120]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "BEIT_SFT"

In [121]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/beit-base-patch16-224")

beit_config = BeitConfig.from_pretrained("microsoft/beit-base-patch16-224")
encoder = BeitModel.from_pretrained("microsoft/beit-base-patch16-224", config=beit_config)

model = BEiTClassifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
print()

In [122]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="kaggle")

# choose optimizer and scheduler (if needed)
# optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [123]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="no", scheduler=scheduler, exp=exp, env="kaggle")

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/denterry/ssl-adversarial/84f706dd8a0c42a5a134b629e275916b

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:04<00:00, 19.13it/s]




 Train:
  Loss: 2.291240727920532, Accuracy: 0.29616
 Validation:
  Loss: 1.5170827505111695, Accuracy: 0.4476
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:04<00:00, 19.57it/s]


...

100%|██████████| 79/79 [00:04<00:00, 19.35it/s]




 Train:
  Loss: 0.9417358040618896, Accuracy: 0.66234
 Validation:
  Loss: 1.0389049693107606, Accuracy: 0.6292
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------

Epoch 5:



100%|██████████| 79/79 [00:04<00:00, 19.49it/s]




 Train:
  Loss: 0.7946329331588745, Accuracy: 0.71808
 Validation:
  Loss: 0.9934544400215148, Accuracy: 0.6515
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : BEIT_SFT
COMET INFO:     url                   : https://www.comet.com/denterry/ssl-adversarial/84f706dd8a0c42a5a134b629e275916b
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (647.2160034179688, 1126774.25)
COMET INFO:     train accuracy [5]                  : (0.29616, 0.71808)
COMET INFO:     train loss (by steps) [1955]        : (0.538834810256958, 19.35883331298828)
COMET INFO:     train loss [5]                      : (0.7946329331588745, 2.291240727920532)
COMET INFO:     validation accuracy (No Attack) [5] : (0.4476, 0.6515)
COMET INFO:     validation loss (No Attack) [5]     : (0.9934544400215

In [124]:
test(model, test_loader, env="kaggle")

100%|██████████| 79/79 [01:16<00:00,  1.03it/s]



Final Metrics:
 No Attack:
   Loss: 0.9934544400215148, Accuracy: 0.6515
 i-FGSM:
   Loss: 9.28764167175293, Accuracy: 0.001
 PGD:
   Loss: 9.180963650512695, Accuracy: 0.001
 DI-FGSM:
   Loss: 7.203410546875, Accuracy: 0.0115
